In [113]:
import os
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D
from keras import backend

In [114]:
backend.clear_session()

In [115]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)

empty_shelf = 0
train_dir = 'data/train'
validation_dir = 'data/validation'
batch_size = 16
num_train_samples = 695
num_validation_samples = 188
num_epochs = 50

In [58]:
kernel = (3, 3)
model = Sequential()
model.add(Conv2D(32, kernel, input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [75]:
rescale_multiplier = 1. / 255
train_datagen = ImageDataGenerator(
    rescale=rescale_multiplier,
    shear_range=0.2,
    zoom_range=0.1,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=rescale_multiplier)

In [60]:
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_gen = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 695 images belonging to 2 classes.
Found 188 images belonging to 2 classes.


In [61]:
model.fit_generator(
    train_gen,
    steps_per_epoch=num_train_samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_gen,
    validation_steps=num_validation_samples // batch_size)

model.save_weights('runs/saved_weights.h5')

Epoch 1/50
43/43 [==============================] - 7s 170ms/step - loss: 0.7523 - accuracy: 0.5434 - val_loss: 0.6589 - val_accuracy: 0.6307
Epoch 2/50
43/43 [==============================] - 8s 181ms/step - loss: 0.6686 - accuracy: 0.6274 - val_loss: 0.6104 - val_accuracy: 0.7791
Epoch 3/50
43/43 [==============================] - 8s 189ms/step - loss: 0.5822 - accuracy: 0.7049 - val_loss: 0.4453 - val_accuracy: 0.7384
Epoch 4/50
43/43 [==============================] - 8s 189ms/step - loss: 0.5551 - accuracy: 0.7657 - val_loss: 0.2527 - val_accuracy: 0.7965
Epoch 5/50
43/43 [==============================] - 8s 197ms/step - loss: 0.4878 - accuracy: 0.7776 - val_loss: 0.3407 - val_accuracy: 0.8663
Epoch 6/50
43/43 [==============================] - 8s 189ms/step - loss: 0.4622 - accuracy: 0.7896 - val_loss: 0.2568 - val_accuracy: 0.8779
Epoch 7/50
43/43 [==============================] - 9s 216ms/step - loss: 0.4796 - accuracy: 0.8115 - val_loss: 0.3420 - val_accuracy: 0.6686
Epoch 

In [103]:
def isEmptyShelf(pred):
    if pred < 0.5:
        print ("Empty shelf")
    else:
        print ("Full shelf")

In [107]:
hidden_sanity_image_empty = 'data/empty_shelf_sanity.jpeg'

img = load_img(hidden_sanity_image_empty, target_size=(img_width, img_height))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)


pred = model.predict(x)[0][0]
isEmptyShelf(pred)

Empty shelf


In [110]:
hidden_sanity_image_full = 'data/full_shelf_sanity.jpg'
img = load_img(hidden_sanity_image_full, target_size=(img_width, img_height))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)


pred = model.predict(x)[0][0]
isEmptyShelf(pred)

Full shelf
